In [1]:
import numpy as np
import pandas as pd
!pip install haversine 
from haversine import haversine


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 최적의 입지 찾기
1. 사용데이터 : 천안시 빵소, 기차역, 지하철, 고속버스터미널, 버스정류장, 주차장, 주유소, 관광지, 숙박시설, 공원, 등산로 시종점
2. 반영 가중치
- 주요 교통시설 : (터미널, 천안역, 천안아산역)과의 거리 (km): 3
- 1 / 빵소와의 거리 (km): 3
- 주차장: 2
- 주변 대중교통 인프라 (버스 정류장, 지하철역, 주유소): 2
- 관광, 공원, 산: 2.5
- 오락, 숙박: 1

In [3]:
df = pd.read_csv('입지선정데이터\빵소+폐건물+역_위경도.csv')
df=df.iloc[:,1:]

In [19]:
# 데이터 불러오기
폐가후보지 = df[df['grp']=='폐건물']
빵집 = df[df['grp']=='빵소']
인프라 = df[df['grp']=='인프라']
관광 = pd.read_excel('입지선정데이터\충청남도 관광 서비스_위경도.xlsx',sheet_name=1)
오락 = pd.read_excel('입지선정데이터\충청남도 오락시설_위경도.xlsx',sheet_name=1)
숙박 = pd.read_excel('입지선정데이터\충청남도 숙박 서비스_위경도.xlsx',sheet_name=1)
편의시설 = pd.read_excel('입지선정데이터\충청남도 편의시설_위경도.xlsx')
주차장 = pd.read_csv('입지선정데이터\천안시_주차장_위경도.csv',encoding='EUC-KR')
주유소 = pd.read_csv('입지선정데이터\천안시_주유소_위경도.csv')
버스정류장 = pd.read_csv('입지선정데이터\천안시_버스정류장_위경도_1km이내_파일압축.csv',encoding='EUC-KR')
공원 = pd.read_csv('입지선정데이터\충청남도_천안시_도시공원정보_위경도.csv',encoding='EUC-KR')
산 = pd.read_csv('입지선정데이터\천안시_산위치.csv')
산 = 산.iloc[:,1:]

In [20]:
# 빵소 및 인프라
빵소_df = df[df['grp'].isin(['빵소'])]
인프라_df = df[df['grp'].isin(['인프라'])]

# 주차장
주차장.rename(columns = {'주차장명':'name','주소':'addr','위도' : 'lat','경도':'long'},inplace =True)
주차장_df = 주차장[['name','addr','lat','long']]
주차장_df['grp'] ='주차장'

#관광
관광_df = 관광[['사업장명','Latitude','Longitude']]
관광_df.rename(columns = {'사업장명':'name','Latitude' : 'lat','Longitude':'long'},inplace =True)
관광_df['grp'] ='관광'

#오락
오락_df = 오락[['사업장명','Latitude','Longitude']]
오락_df.rename(columns = {'사업장명':'name','Latitude' : 'lat','Longitude':'long'},inplace =True)
오락_df['grp'] ='오락'

#버스정류장
버스정류장_df = 버스정류장[['정류장명','위도','경도']]
버스정류장_df.rename(columns = {'정류장명':'name','위도' : 'lat','경도':'long'},inplace =True)
버스정류장_df['grp'] = '정류장'

#숙박
숙박_df = 숙박[['사업장명','Latitude','Longitude']]
숙박_df.rename(columns = {'사업장명':'name','Latitude' : 'lat','Longitude':'long'},inplace =True)
숙박_df['grp'] ='숙박'

#주유소
주유소_df = 주유소[['상호','Latitude','Longitude']]
주유소_df.rename(columns = {'상호':'name','Latitude' : 'lat','Longitude':'long'},inplace =True)
주유소_df['grp'] ='주유소'

#공원
공원_df = 공원[['공원명','위도','경도']]
공원_df.rename(columns = {'공원명':'name','위도' : 'lat','경도':'long'},inplace =True)
공원_df['grp'] ='공원'

#산
산_df = 산[['name','lat','long']]
산_df['grp'] ='산'

total_df = pd.concat([빵소_df,인프라_df,공원_df,주차장_df,관광_df,오락_df,버스정류장_df,숙박_df,주유소_df,산_df],axis =0 )

C:\Users\LG\AppData\Local\Temp\ipykernel_8320\1082008990.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주차장_df['grp'] ='주차장'
C:\Users\LG\AppData\Local\Temp\ipykernel_8320\1082008990.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  관광_df.rename(columns = {'사업장명':'name','Latitude' : 'lat','Longitude':'long'},inplace =True)
C:\Users\LG\AppData\Local\Temp\ipykernel_8320\1082008990.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [21]:
폐가_points = np.array([list(i) for i in zip(폐가후보지['lat'], 폐가후보지['long'])])
폐가_points

# 빵소 3
빵소_points = np.array([list(i) for i in zip(빵소_df['lat'], 빵소_df['long'])])

# 주요교통 : 3
주요교통_df = 인프라_df
주요교통_points = np.array([list(i) for i in zip(인프라_df['lat'], 인프라_df['long'])])

# 관광, 공원, 산: 2.5
관광자연_df = pd.concat([관광_df,공원_df,산_df],axis = 0)
관광자연_points = np.array([list(i) for i in zip(관광자연_df['lat'], 관광자연_df['long'])])

# 교통: 역(두정역, 봉명역,쌍용역), 버스정류장, 주유소,주차장 2
교통_df = pd.concat([버스정류장_df,주유소_df, 주차장_df],axis = 0)
교통_points = np.array([list(i) for i in zip(교통_df['lat'], 교통_df['long'])])

# 오락, 숙박 1
오락숙박_df = pd.concat([오락_df,숙박_df],axis = 0)
오락숙박_points = np.array([list(i) for i in zip(오락숙박_df['lat'], 오락숙박_df['long'])])

# 전체
total_points = np.array([list(i) for i in zip(total_df['lat'], total_df['long'])])

- 비율반영

In [22]:
# 가중치
전체w = total_points.shape[0]
빵소w = 빵소_df.shape[0]
주요교통w = 주요교통_df.shape[0]
관광자연w = 관광자연_df.shape[0]
교통w = 교통_df.shape[0]
오락숙박w = 오락숙박_df.shape[0]

빵소1 = 0.140*(전체w-빵소w) / 전체w 
주요교통1 = 0.140*(전체w-주요교통w) / 전체w 
관광자연1 = 0.116*(전체w-관광자연w) / 전체w 
교통1 = 0.093*(전체w-교통w) / 전체w 
오락숙박1 = 0.047*(전체w-오락숙박w) / 전체w 

In [23]:
# 비율 반영


def pmedian2(폐가_points, 빵소_points, 주요교통_points, 관광자연_points, 교통_points, 오락숙박_points):
    ### 빵소
    havers1 = []
    for i in 폐가_points:
        site1 = []
        for j in 빵소_points:
            site1.append(haversine(i,j)*1000)
        havers1.append(site1)

    location = list(폐가후보지['name'])
    location1 = list(빵소_df['name'])

    havers_D1 = dict(zip(location, [dict(zip(location1, i)) for i in havers1]))
    D1 = pd.DataFrame(havers_D1)
    
    최소값 = list(D1.min(axis=1))

    # 최단거리인 경우 가중치 부여, 아니면 0
    idx = D1.index
    col = D1.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D1.loc[idx[i], col[j]] == 최소값[i]:
                D1.loc[idx[i], col[j]] = 빵소1 # 가중치
            else:
                D1.loc[idx[i], col[j]] = 0
    ### 주요교통
    havers2 = []
    for i in 폐가_points:
        site2 = []
        for j in 주요교통_points:
            site2.append(haversine(i,j)*1000)
        havers2.append(site1)

    location2 = list(주요교통_df['name'])

    havers_D2 = dict(zip(location, [dict(zip(location2, i)) for i in havers2]))
    D2 = pd.DataFrame(havers_D2)
    
    최소값 = list(D2.min(axis=1))

    # 최단거리인 경우 가중치 부여, 아니면 0
    idx = D2.index
    col = D2.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D2.loc[idx[i], col[j]] == 최소값[i]:
                D2.loc[idx[i], col[j]] = 주요교통1 # 가중치
            else:
                D2.loc[idx[i], col[j]] = 0
    ### 자연관광
    havers3 = []
    for i in 폐가_points:
        site3 = []
        for j in 관광자연_points:
            site3.append(haversine(i,j)*1000)
        havers3.append(site3)

    location3 = list(관광자연_df['name'])

    havers_D3 = dict(zip(location, [dict(zip(location3, i)) for i in havers3]))
    D3 = pd.DataFrame(havers_D3)
    
    최소값 = list(D3.min(axis=1))

    idx = D3.index
    col = D3.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D3.loc[idx[i], col[j]] == 최소값[i]:
                D3.loc[idx[i], col[j]] = 관광자연1 # 가중치
            else:
                D3.loc[idx[i], col[j]] = 0
    ### 교통
    havers4 = []
    for i in 폐가_points:
        site4 = []
        for j in 교통_points:
            site4.append(haversine(i,j)*1000)
        havers4.append(site4)

    location4 = list(교통_df['name'])

    havers_D4 = dict(zip(location, [dict(zip(location4, i)) for i in havers4]))
    D4 = pd.DataFrame(havers_D4)
    
    최소값 = list(D4.min(axis=1))

    idx = D4.index
    col = D4.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D4.loc[idx[i], col[j]] == 최소값[i]:
                D4.loc[idx[i], col[j]] = 교통1 # 가중치
            else:
                D4.loc[idx[i], col[j]] = 0
    ### 오락숙박
    havers5 = []
    for i in 폐가_points:
        site5 = []
        for j in 오락숙박_points:
            site5.append(haversine(i,j)*1000)
        havers5.append(site5)

    location5 = list(오락숙박_df['name'])

    havers_D5 = dict(zip(location, [dict(zip(location5, i)) for i in havers5]))
    D5 = pd.DataFrame(havers_D5)
    
    최소값 = list(D5.min(axis=1))

    idx = D5.index
    col = D5.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D5.loc[idx[i], col[j]] == 최소값[i]:
                D5.loc[idx[i], col[j]] = 오락숙박1 # 가중치
            else:
                D5.loc[idx[i], col[j]] = 0  
    D_final = pd.concat([D1, D2, D3, D4, D5])
    # return D_final
    print(D_final.sum().sort_values(ascending=False))     

In [24]:
pmedian2(폐가_points, 빵소_points, 주요교통_points, 관광자연_points, 교통_points, 오락숙박_points)

두정동 폐공사장    21.586989
삼룡동 폐건물     16.363022
쌍용동 폐공장     15.990309
dtype: float64


In [25]:
!pip install folium
import folium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
map = folium.Map(location=[36.79,127.12],tiles="cartodb positron")

In [27]:
folium.Marker([36.8319745, 127.1385192],popup='두정동 폐공사장').add_to(map)
folium.Marker([36.7963716, 127.1293773],popup='쌍용동 폐공장').add_to(map)
folium.Marker([36.7901058, 127.1644011],popup='삼룡동 폐건물').add_to(map)

In [29]:
for loc in 빵소_points:
    folium.Marker(loc,icon=folium.Icon(color='red', icon='star')).add_to(map)

In [30]:
for loc in 주요교통_points:
    folium.CircleMarker(loc,
                        radius=5,
                        color = 'brown',
                        # fill_color = 'brown',
                        opacity=0.7).add_to(map)
for loc in 관광자연_points:
    folium.CircleMarker(loc,
                        radius=5,
                        color = 'green',
                        # fill_color = 'green',
                        opacity=0.7).add_to(map)
for loc in 교통_points:
    folium.CircleMarker(loc,
                        radius=5,
                        color = 'blue',
                        # fill_color = 'blue',
                        opacity=0.7).add_to(map)
for loc in 오락숙박_points:
    folium.CircleMarker(loc,
                        radius=5,
                        color = 'purple',
                        # fill_color = 'purple',
                        opacity=0.7).add_to(map)

In [227]:
map